In [ ]:
import pandas as pd
import numpy as np
import os
import json
import seaborn as sns, matplotlib.pyplot as plt
# from jsonschema import validate, ValidationError

import warnings; warnings.filterwarnings('ignore')

# Display all columns in dataframes

pd.set_option('display.max_columns', None)

dataset_path = "G:\\My Drive\\Study\\Projects - Data\\Data Projects - Analysis for Fun 1\\data\\datasets-property-click-prediction"


# Function for Checking NULL Counts and NULL Percentages
# ======================================================
def check_null(df):
        
    null_counts = pd.DataFrame(df.isnull().sum(), columns=['Null_Count'])
    null_percentage = pd.DataFrame(round((df.isnull().sum()/len(df))*100, 2), columns=['Null_Percentage'])
    
    # Combining the two dataframes
    null_info = pd.concat([null_counts, null_percentage], axis = 1)

    # Filtering to show only rows where Null_Count > 0
    null_info = null_info[null_info['Null_Count'] > 0]
    
    return null_info   

## Data Reading & Pre-processing

### Reading and Pre-processing the `property_details dataset`

In [ ]:
property_data = pd.read_csv(os.path.join(dataset_path, "property_data_set.csv"))
print("\n",property_data.shape)
display(property_data.head(2))

In [ ]:
property_data.info()

In [ ]:
# First thing we notice that the date column is in string format. We need to convert it to datetime.
property_data['activation_date'] = pd.to_datetime(property_data['activation_date'], format='mixed') # ormat='ISO8601
property_data.head()

In [ ]:
property_data.isnull().sum()

In [ ]:
# We can see that there are missing values in 3 columns: 'total_floor', 'building_type', and 'pin_code'.
# We will fill these missing values later.

In [ ]:
# We can change some more data types to optimize memory usage.
# We use a way to handle missing values as well while converting data types - using 'coerce'.

property_data['total_floor'] = pd.to_numeric(property_data['total_floor'], errors='coerce').astype('Int16')
property_data['pin_code'] = property_data['pin_code'].astype('string')

In [ ]:
property_data.isnull().sum()

In [ ]:
property_data[['total_floor', 'building_type', 'pin_code']].info()

In [ ]:
property_data['building_type'].mode()[0]

In [ ]:
property_data['building_type'].unique()

In [ ]:
# Now we will get back to the missing values.
# Since the missing values are meagre.
# Hence we fill categorical columns with mode values and numerical columns with median values.

# Categorical Imputation
property_data['building_type'] = property_data['building_type'].fillna(property_data['building_type'].mode()[0])
property_data['pin_code'] = property_data['pin_code'].fillna(property_data['pin_code'].mode()[0])

# Numerical Imputation
property_data['total_floor'] = property_data['total_floor'].fillna(property_data['total_floor'].median())

# Now let's check if there are any more missing values in the dataset.
property_data.isnull().sum()

In [ ]:
# We successfully cleaned the dataset.
# We will explore more on this dataset later when we have merged all the datsets.

#  We save the cleaned dataset to a new csv file.
property_data.to_csv(os.path.join(dataset_path, "property_data_processed.csv"), index=False)

# Now let's check the further datasets.

### Reading and Pre-Processing the `property_photos dataset`

In [ ]:
property_photos = pd.read_csv(os.path.join(dataset_path, "property_photos.tsv"), sep='\t')
print("\n",property_photos.shape)
display(property_photos.head(2))

In [ ]:
property_photos.isnull().sum()

In [ ]:
# 
# Our end goal is to get the count of photos for each property.
# The photos are stored as a string in the 'photo_urls' column. They are json formatted strings.
# But first, let's correct the corrupted json values in the 'photo_urls' column.
# Once the json values are corrected, we can count the number of photos.
# 

# What NULLs signify??
# ------------------
# A null value or "nan" values in 'photo_urls' column means that the property has no photos, so will contain a zero photo count.
# 

In [ ]:
def correct_json(x):
    # if value is null put count with 0 photos
    if x is np.nan or x == 'NaN':
        return 0
    else :
        # Replace corrupted values then convert to json and get count of photos
        return len(json.loads( x.replace('\\' , '').replace('{title','{"title').replace(']"' , ']').replace('],"', ']","') ))
    
property_photos['photo_count'] = property_photos['photo_urls'].apply(correct_json)

# drop the 'photo_urls' column
property_photos.drop('photo_urls', axis=1, inplace=True)

display(property_photos.head())

# Saving the cleaned dataset to a new csv file.
property_photos.to_csv(os.path.join(dataset_path, "property_photos_processed.csv"), index=False)

### Reading & Pre-processing the `property_interactions dataset`

In [ ]:
# Reading the `property_interactions dataset`
# -------------------------------------------
property_interactions = pd.read_csv(os.path.join(dataset_path, "property_interactions.csv"))
print("\n",property_interactions.shape)
display(property_interactions.head(2))

In [ ]:
# For this case we want to count the number of unique interactions for each property_id.
# Also we would want to count interactions in the first 3 days, 5 days, and 7 days respectively.

# Hence for each property_id, we will also need listing creation date "activation_date", which is not available in this dataset.
# We use the property details dataset for this purpose.

property_data = pd.read_csv(os.path.join(dataset_path, "property_data_processed.csv"))

# We now merge the property_interactions dataset with the property_data dataset on property_id.
# But from the property_data dataset, we get the 'activation_date' column.

property_interactions = property_interactions.merge(property_data[['property_id', 'activation_date']], on='property_id')
property_interactions.head()

In [ ]:
property_interactions.info()

In [ ]:
# Now we see that the 'request_date' and 'activation_date' are NOT in datetime format.
# So, we convert these columns to datetime format.

property_interactions['request_date'] = pd.to_datetime(property_interactions['request_date'], format='mixed')
property_interactions['activation_date'] = pd.to_datetime(property_interactions['activation_date'], format='mixed')

In [ ]:
property_interactions.info()

In [ ]:
# We see that the columns are now in datetime format.

In [ ]:
property_interactions['property_id'].nunique()

In [ ]:
# What I need to do is generate the total interactions - counting all the property listing clicks for each unique property_id
# Also, I need to generate the total interactions occurred withing 3, 5, and 7 days respectively of the activation of the listing.

In [ ]:
# Function to count interactions within a given number of days from activation
def count_interactions_within_days(group, days):
    end_date = group['activation_date'].iloc[0] + pd.Timedelta(days=days)
    return group[(group['request_date'] >= group['activation_date'].iloc[0]) & (group['request_date'] < end_date)].shape[0]

# Group by property_id and calculate the required metrics
result = property_interactions.groupby('property_id').apply(lambda x: pd.Series({
    'total_interactions': x.shape[0],
    'count_3_days': count_interactions_within_days(x, 3),
    'count_5_days': count_interactions_within_days(x, 5),
    'count_7_days': count_interactions_within_days(x, 7)
})).reset_index()

display(result.head())

In [ ]:
print(result.shape)
result.to_csv(os.path.join(dataset_path, "property_interactions_processed.csv"), index=False)

### Merging the 3 datasets

In [ ]:
# Re-Reading the cleaned datasets

df1 = pd.read_csv(os.path.join(dataset_path, "property_data_processed.csv"))
df2 = pd.read_csv(os.path.join(dataset_path, "property_photos_processed.csv"))
df3 = pd.read_csv(os.path.join(dataset_path, "property_interactions_processed.csv"))

# Merging the property_data_processed and property_photos_processed datasets on property_id
property_merged = df1.merge(df2, on='property_id').merge(df3, on='property_id',  how='left')

print(property_merged.shape)
display(property_merged.head())

In [ ]:
# property_merged.isnull().sum()

In [ ]:
# Saving the merged dataset to a new csv file.
property_merged.to_csv(os.path.join(dataset_path, "merged_property_data.csv"), index=False)

In [ ]:
# Now we have the a complete and merged dataset.
# So we will begin further processing and analyzing this dataset.